# load adult dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

DEMO_URL = 'http://ctgan-demo.s3.amazonaws.com/census.csv.gz'


def load_demo():
    """Load the demo."""
    return pd.read_csv(DEMO_URL, compression='gzip')

adult = load_demo()
adult_label = adult.iloc[:,-1]
adult = adult.iloc[:,:-1]

# CTGAN

In [ ]:
!pip install ctgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2

In [ ]:
import pandas as pd
from ctgan import CTGAN

def train_ctgan(data, discrete_columns):
# Initialize the CTGAN model
  ctgan = CTGAN(cuda=True, verbose=True)
  # Specify categorical columns
  # discrete_columns = []
  # Train the model
  ctgan.fit(data, discrete_columns)

  return ctgan

def synthesize_data(ctgan, sample_size):
  return ctgan.sample(n=sample_size)

In [ ]:
adult_discrete = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]
ctgan_model = train_ctgan(adult, adult_discrete)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
Gen. (-0.48) | Discrim. (0.08): 100%|██████████| 300/300 [10:53<00:00,  2.18s/it]


In [ ]:
adult_ctgan = synthesize_data(ctgan_model, 100000)
adult_ctgan.to_csv('adult_ctgan_100k_onlyX.csv', index=False)

In [ ]:
adult_ctgan.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,37,Private,202528,Some-college,10,Separated,Protective-serv,Not-in-family,Black,Female,7,0,47,United-States,<=50K
1,34,Private,176828,HS-grad,9,Divorced,Adm-clerical,Other-relative,White,Female,-20,0,49,United-States,<=50K
2,52,Self-emp-not-inc,169759,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,-25,-1,33,United-States,<=50K
3,24,Private,100468,Bachelors,13,Never-married,Adm-clerical,Unmarried,White,Female,-12,1,24,United-States,<=50K
4,68,Private,105342,Doctorate,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,7836,0,60,United-States,>50K


### save trained model

In [ ]:
ctgan_model.save('/content/drive/Shareddrives/DA Summer Research/ctgan trained model/adults_ctgan_onlyX.pkl')

### load ctgan from saved model

In [ ]:
import pickle
from ctgan import CTGAN

# Load the model from the file
model_path = '/content/drive/Shareddrives/DA Summer Research/ctgan trained model/adults_ctgan.pkl'
with open(model_path, 'rb') as f:
    ctgan = pickle.load(f)

# SDV

In [ ]:
!pip install sdv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Usi

In [ ]:
def get_metadata(data, verbose=False):
  from sdv.metadata import SingleTableMetadata

  metadata = SingleTableMetadata()
  metadata.detect_from_dataframe(data)
  metadata_dict = metadata.to_dict()
  if verbose:
    print(metadata_dict)
    metadata.visualize(
        show_table_details='summarized',
        output_filepath='my_metadata.png'
    )

  return metadata

def create_synthesizer(metadata):
  from sdv.single_table import CTGANSynthesizer

  synthesizer = CTGANSynthesizer(
      metadata, # required
      # enforce_rounding=True,
      # epochs=500,
      verbose=True,
      cuda=True
  )

  return synthesizer

def train_synthesizer(data, verbose=False):
  metadata = get_metadata(data)
  synthesizer = create_synthesizer(metadata)
  synthesizer.auto_assign_transformers(data)
  synthesizer.fit(data)
  if verbose:
    fig = synthesizer.get_loss_values_plot()
    fig.show()

  return synthesizer

In [ ]:
synthesizer = train_synthesizer(adult)

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:86: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Gen. (0.00) | Discrim. (0.00):   0%|          | 0/300 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Gen. (-0.51) | Discrim. (-0.22): 100%|██████████| 300/300 [10:42<00:00,  2.14s/it]


In [ ]:
adult_sdv = synthesizer.sample(num_rows=100_000)
adult_sdv.to_csv('adult_sdv_100k_onlyX.csv')

In [ ]:
synthesizer.save(
    filepath='/content/drive/Shareddrives/DA Summer Research/sdv trained model/adult_synthesizer_onlyX.pkl'
)

# one hot encoding

In [ ]:
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder

In [ ]:
xtest = pd.read_csv("/content/drive/Shareddrives/DA Summer Research/adult_sdv_100k_onlyX.csv")

In [ ]:
adult_sdv = pd.read_csv("/content/drive/Shareddrives/DA Summer Research/adult_sdv_100k_onlyX.csv")

In [ ]:
# Prepend column names to values
def prepend_col_name(df, columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: f"{col}_{x}")
    return df

# Prepend column names to xtrain and xtest
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
numerical_cols = list(set(adult.columns) - set(categorical_columns))
xtrain = prepend_col_name(adult, categorical_columns)
xtest = prepend_col_name(xtest, categorical_columns)

# Prepare xtrain_prep and xtest_prep
xtrain_prep = xtrain[numerical_cols]
xtest_prep = xtest[numerical_cols]

# One-Hot Encoding with modified categorical values
for col in categorical_columns:
    print(col)

    # One-Hot Encoding for xtrain
    xtrain_onehot = OneHotEncoder().fit_transform(xtrain[col].values.reshape(-1,1))
    xtrain_onehot = xtrain_onehot.toarray()
    xtrain_onehot = pd.DataFrame(xtrain_onehot, columns = xtrain[col].unique())
    xtrain_prep = pd.concat([xtrain_prep, xtrain_onehot], axis=1)

    # One-Hot Encoding for xtest
    xtest_onehot = OneHotEncoder().fit_transform(xtest[col].values.reshape(-1,1))
    xtest_onehot = xtest_onehot.toarray()
    xtest_onehot = pd.DataFrame(xtest_onehot, columns = xtest[col].unique())
    xtest_onehot = xtest_onehot.set_index(xtest.index)
    xtest_prep = pd.concat([xtest_prep, xtest_onehot], axis=1)

    # Check differences between xtrain and xtest
    dif1 = set(xtest[col].unique()) - set(xtrain[col].unique())
    print(len(dif1), dif1)

# Output the prepared dataframes
# print("xtrain_prep:\n", xtrain_prep)
# print("xtest_prep:\n", xtest_prep)

workclass
0 set()
education
0 set()
marital-status
0 set()
occupation
0 set()
relationship
0 set()
race
0 set()
sex
0 set()
native-country
0 set()


# Gaussian Naive Bayes

In [ ]:
xtrain_prep = xtrain_prep.reindex(sorted(xtrain_prep.columns), axis=1)
xtrain_prep

,age,capital-gain,capital-loss,education-num,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,...,sex_ Male,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay
0,39,2174,0,13,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,50,0,0,13,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,38,0,0,9,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,0,0,13,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,0,0,12,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
32557,40,0,0,9,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
32558,58,0,0,9,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
32559,22,0,0,9,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
xtest_prep = xtest_prep.reindex(sorted(xtest_prep.columns), axis=1)
xtest_prep

,age,capital-gain,capital-loss,education-num,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,...,sex_ Male,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay
0,55,3,0,14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,58,27,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,30,20,1,11,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,30,17,2,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,32,16,0,11,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,40,16,0,11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
99996,68,35,1,14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
99997,35,16,0,8,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
99998,43,14,0,15,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
# X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB().fit(xtrain_prep, adult_label)
adult_sdv_label = gnb.predict(xtest_prep)

adult_sdv_label = pd.DataFrame(adult_sdv_label, columns = ['income'])
adult_sdv_gaussian = pd.concat([adult_sdv, adult_sdv_label], axis=1)
adult_sdv_gaussian.to_csv('adult_sdv_gaussian_100k.csv')

# print("Number of mislabeled points out of a total %d points : %d"
#       % (X_test.shape[0], (y_test != y_pred).sum()))

# Categorical Naive Bayes

In [ ]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB().fit(xtrain_prep, adult_label)
adult_sdv_label = cnb.predict(xtest_prep)

adult_sdv_label = pd.DataFrame(adult_sdv_label, columns = ['income'])
adult_sdv_categorical = pd.concat([adult_sdv, adult_sdv_label], axis=1)
adult_sdv_categorical.to_csv('adult_sdv_categorical_100k.csv')